In [ ]:
#Basic notebook with a few juypter examples


# Metrics to calc

# Users that haven't purchased in a while
# Last purchase per user - any product

#Last purchase per user - each product
#  * somehow include no purchases - for ones that they have been assigned


In [5]:
import pandas as pd
import logging
from datetime import date, timedelta
import numpy as np
import seaborn as sns


logging.basicConfig(level=logging.DEBUG, format="%(message)s")
LOGGER = logging.getLogger(__name__)
LOGGER.debug("log level = %s", str(LOGGER.getEffectiveLevel()))


ModuleNotFoundError: No module named 'pandas'

In [6]:
DATA_PATH="./data/purchases.csv"
DATE_AS_OF=date.fromisoformat('2023-06-01') #Date for analysis
#For data visualization - date range sizes
DATE_GROUP_DAYS=30
CUTOFF_DAYS=90



REQUIRED_COLUMNS=['user', 'date','product', 'price']

#loist of recommended actions for different date threseholds
ACTION_DATES = dict(out_of_warranty=30, obsolete=90)


NameError: name 'date' is not defined

In [7]:

def clean_raw(df:pd.DataFrame, required_columns:[str])->pd.DataFrame:
    """
    Check and clean purchases dataframe.  
    :param df: 
    :return:    updated dataframe  or exception if issue. 
    """
    
    missing_cols = set(required_columns) - set(df.columns)
    if missing_cols:
        raise Exception(f"Invalid dataframe Missing required columns: {', '.join(missing_cols)}")
    
    #trim up - sometimes there are bad rows at the bottom of the csv. We use the first required column
    df = df.loc[df[required_columns[0]].notnull()]
    
    #Error if any of the cells are blanks
    if df.isnull().sum().sum():
        raise Exception(f"Invalid dataframe: some cells are empty")

    #convert dates
    df['date']  = pd.to_datetime(df['date'], format='%Y-%m-%d').dt.date
     
    return df


NameError: name 'pd' is not defined

In [ ]:
df_all = pd.read_csv(DATA_PATH)

df_all=clean_raw(df_all, required_columns=REQUIRED_COLUMNS)
LOGGER.info("Loaded %d clean rows from CSV[%s]", len(df_all), DATA_PATH)

#Add number of days since date.   Makes some downstream calcs a little easier
df_all['days_ago'] = df_all['date'].apply(lambda x: (DATE_AS_OF - x).days)

# Count unique values in each column
for col in ['user', 'product']: 
    LOGGER.info("  # unique values in col: %s = %d",col, df_all[col].nunique())

LOGGER.info("Date-range min: %s", df_all['date'].min())
LOGGER.info("Date-range max: %s", df_all['date'].max())


In [ ]:
df_all.head(5)

In [ ]:
# Begin to count the recommend actions. 
#sort so the largest date value is first in dict.  i.e., we want to recommend the action with the largest date range
actions = {k: v for k, v in sorted(ACTION_DATES.items(), key=lambda item: item[1], reverse=True)} 
default='ok'


def determine_action(days:int) ->str: 
    for action, d_thresehold in actions.items():
        if days >= d_thresehold:
            return action
    return default

df_all['action'] = df_all['days_ago'].apply(determine_action)
df_all.head(5)

#Find the ones beyond the dates. 

In [ ]:

# sort the actions by oldest


In [ ]:
#####################################
#most recent purchases per user - any product
df_recent = df_all.sort_values(by='date', ascending=False)

# group the data by Name and get the first row of each group
df_most_recent_purchases = df_recent.groupby('user').first()

df_most_recent_purchases


In [ ]:
#####################################
#most recent purchases per user - any product

df_recent = df_all.sort_values(by='date', ascending=False)

# group the data by Name and get the most recent purchase (first row) of each group
df_most_recent_purchase = df_recent.groupby(['User'],as_index=False).first()
df_most_recent_purchase.head(5)

In [ ]:
#####################################
#most recent purchases per user - list all products

df_recent = df_all.sort_values(by='date', ascending=False)

# group the data by Name and get the most recent purchase (first row) of each group
df_most_recent_purchases_all = df_recent.groupby(['user', 'product'],as_index=False).first()
df_most_recent_purchases_all.head(15)

In [ ]:
### Find purchases that are old - such as out of warranty date.
filtered_df = df_most_recent_purchases_all.loc[df_most_recent_purchases_all['date'] < '2023-02-01']


print(f"Num purchases out of warranty: {len(filtered_df)}")
filtered_df.head(5)


In [ ]:
#####################################
# Last purchase per user per product
#
# Note: this doesn't handle case where a user never purchased
#####################################
#Generate blocks of cutoff days.    The first is the max.
dates = []
dateblocks = {}

dmin = df_all['date'].min().date()
print(f"dmin: Earliest entry from date: {dmin}")
d = DATE_AS_OF
ndays = 0
while d >= dmin:
 dates.insert(0,d)

 prevdate = d - timedelta(days=DATE_GROUP_DAYS)
 ndays_max=ndays + DATE_GROUP_DAYS-1
 dateblocks[ndays] = dict(ndays=ndays,
                          date=d,
                          prevdate=prevdate,
                          ndays_max = ndays_max,
                          label=f"{ndays} to {ndays_max} days")

 #Next block
 d = prevdate
 ndays += DATE_GROUP_DAYS

 #print(f"date slice: {d}")
 #print(f"dateblocks: {dateblocks}")

 # End up with a dict

 #values
 #* n days
 #* label
 #df (slice) of entries that match.
 #* be able to visualize.


 #<= 10 days:   15
 #11 <= 20 days  37
 #21

In [ ]:
#dateblocks

In [ ]:
df_all['days_since_login'] = df_all['date'].apply(lambda d: (DATE_AS_OF - d.date()).days)

def calc_date_block(ndays):

    for d in sorted(dateblocks.keys()):
        #print(f"calc_date_block d={str(d)}  v={dateblocks.get(d)}")
        v = dateblocks[d]
        if ndays <= v.get('ndays_max'):
            return v.get('label')
    return ('unknown')

df_all['days_since_label'] = df_all['days_since_login'].apply(calc_date_block)

df_all.head(5)

In [ ]:
df_date_bands = df_all.groupby('days_since_label',as_index=False).count()
df_date_bands.head(5)

In [ ]:


sns.barplot(x='days_since_label', y='User', data=df_date_bands) #hue='variable')
#plt.xticks(rotation=90)
#plt.ylabel('Returns')
#plt.title('Portfolio vs Benchmark Returns');

In [ ]:


df = filtered_df
df

 Filter data between two dates
filtered_df = df.loc[(df['date'] >= '2020-09-01')
                     & (df['date'] < '2020-09-15')

In [ ]:
df_all['date'].min()

In [ ]:
my_dict = {'a': 3, 'b': 1, 'c': 2}
sorted_dict = {k: v for k, v in sorted(my_dict.items(), key=lambda item: item[1], reverse=True)}  # Sort by values
print(sorted_dict)  # Output: {'b': 1, 'c': 2, 'a': 3}
